In [15]:
# library
import pandas as pd
# library
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 39.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [16]:
data = pd.read_csv(r'/content/Group_4.csv')
data.shape

(46, 1)

In [17]:
data.head()

review_text
0  I won't write much about the quality of sound ...
1  After sales service is very poor. My Tabs scre...
2  the XB910N is a major downgrade from the previ...
3  Always use this I liked this product works goo...
4  The T shirt s good for sports or exercise purp...

In [18]:
# Import necessary libraries
import spacy
import string
import pickle

# Load Spacy model
nlp = spacy.load('en_core_web_sm')

def preprocessing(sentence):
    # Create a Doc object
    doc = nlp(sentence)

    # Generate list of tokens
    tokens = [token.text for token in doc]

    # Remove stop words and punctuation, and perform lemmatization
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    text = ' '.join(i for i in tokens)

    return text

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import dump, load

# Preprocess your text data
data['review_text'] = data['review_text'].apply(preprocessing)

X = data['review_text']

#load the fitted tfidf vectorizer from disk
with open('/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/tfidf_vectorizer.pkl', 'rb') as handle:
    loaded_vectorizer = pickle.load(handle)

# Use the loaded vectorizer to transform your test dataset
X_tfidf = loaded_vectorizer.transform(X)

In [20]:
# load trained models
n = 100
clf_rf = load(f'/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/clf_rf_{n}.joblib')
# Perform inference
data['result'] = clf_rf.predict(X_tfidf)
data.shape

(46, 2)

In [21]:
data.head()

review_text  result
0  will write quality sound product true advertis...       5
1  sale service poor tabs screen damage visit 3 t...       1
2  xb910n major downgrade previous version xb ser...       1
3                    use like product work good skin       5
4  t shirt s good sport exercise purpose wear nor...       5

In [22]:
data[data.result==1].head()

review_text  result
1   sale service poor tabs screen damage visit 3 t...       1
2   xb910n major downgrade previous version xb ser...       1
7   genuine review 2 day usually spend 2k headphon...       1
12  buy samsung service buy wache machine offline ...       1
17  phone house jio 5 g airtel 5 g network device ...       1

In [24]:
data.result.value_counts()

result
5    26
1     8
4     6
2     3
3     3
Name: count, dtype: int64

In [23]:
data[['review_text','result']].to_csv('/content/Group_4_m.csv')